# ManuL

### Data preprocessing

Загрузка необходимых библиотек

In [54]:
import pandas as pd
import numpy as np
import re

Загрузка данных

In [55]:
df = pd.read_excel('../data/raw/UTexas Aptamer Database.xlsx')

Преобразуем колонки к snake_case

In [56]:
def to_snake_case(name):
    name = re.sub(r'[^\w\s]', ' ', name)
    name = re.sub(r'\s+', ' ', name)
    words = name.split()
    return '_'.join(word.lower() for word in words)

df.columns = [to_snake_case(col) for col in df.columns]

Просмотрим информацию о датасете

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 28 columns):
 #   Column                                                                                 Non-Null Count  Dtype  
---  ------                                                                                 --------------  -----  
 0   year_of_paper                                                                          1495 non-null   int64  
 1   link_to_pubmed_entry                                                                   1495 non-null   object 
 2   journals                                                                               1495 non-null   object 
 3   journal_doi                                                                            1495 non-null   object 
 4   citation                                                                               1495 non-null   object 
 5   type_of_nucleic_acid                                                        

Выведем 10 случайных строк

In [58]:
df.sample(10)

,year_of_paper,link_to_pubmed_entry,journals,journal_doi,citation,type_of_nucleic_acid,name_of_aptamer,target,aptamer_sequence,sequence_length,...,ph,molecular_weight_of_target,application_as_quoted_in_the_referenced_paper,post_selex_modifications_to_the_aptamer,additional_information,serial_number,parent_sequence_serial_number,corresponding_author_name_email_address,please_fill_out_the_form_for_any_feedbacks_comments,aptagen_cross_referencing_check_aptamer_chemistry_affinity_length_gc_content_sequence
635,2006,https://pubmed.ncbi.nlm.nih.gov/16940097/,Antimicrob Agents Chemother,https://doi.org/10.1128/AAC.01603-05,"Jones, L. A., Clancy, L. E., Rawlinson, W. D.,...",ssDNA,r10/43,RNA-dependent RNA polymerase (RdRp) (nonstruct...,5′GGGAGACAAGAATAAACGCTCAAGGGCGTGGTGGGTGGGGTACT...,83,...,8.0,Not reported,"Therapeutic: "" Research into antiviral agents ...",Not applicable,NaN,10000605,NaN,"White, P. A, p.white@unsw.edu.au.",https://forms.gle/n4TzuyddXQrHYJXF9,NaN
1238,2017,https://pubmed.ncbi.nlm.nih.gov/29245156/,PLoS One,https://doi.org/10.1371/journal.pone.0189558,"Alshaer, W., Ababneh, N., Hatmal, M., Izmirli,...",ssDNA,Ep1,Epithelial cell adhesion molecule (EpCAM),5'GGGATGGATCCAAGCTTACTGGCTCATTGGGTGGTGGCCTCTGG...,90,...,7.4,33–40 kDa,"Diagnostic and Drug Delivery: "" Epithelial cel...",Not applicable,NaN,10001196,NaN,"Alshaer, W, walhan.alshaer@ju.edu.jo",https://forms.gle/n4TzuyddXQrHYJXF9,NaN
402,2002,https://pubmed.ncbi.nlm.nih.gov/11919187/,J Biol Chem,https://doi.org/10.1074/jbc.M111859200,"Ulrich, H., Magdesian, M. H., Alves, M. J., & ...",2'-fluoro-RNA,Aptamer HS6,"Receptor fibronectin, Trypanosoma cruzi (T. cr...",5′ACCGAGUCCAGAAGCUUGUAGUACUAUCGCUACUGCGCCGGUUG...,107,...,7.4,Not reported,"Therapeutic: "" Select RNA aptamers that bind t...",Not applicable,Two′-fluoro-pyrimidines (2′F-dCTP and 2′F-dUTP...,10000382,NaN,"Colli, W, walcolli@usp.br",https://forms.gle/n4TzuyddXQrHYJXF9,5'rApfCpfCprGprAprGpfUpfCpfCprAprGprAprAprGpfC...
832,2010,https://pubmed.ncbi.nlm.nih.gov/20110884/,Molecules,https://doi.org/10.3390/molecules15010215,"Nonaka Y, Sode K, Ikebukuro K. Screening and i...",ssDNA,V7t1,Receptor-binding domain (RBD) of vascular endo...,5'TGTGGGGGTGGACGGGCCGGGTAGA3',25,...,8.0,Not reported,"Biosensors: "" With the aim of creating a more ...",Not applicable,truncated version of Vap7,10000799,NaN,"Ikebukuro K, ikebu@cc.tuat.ac.jp",https://forms.gle/n4TzuyddXQrHYJXF9,5'dTpdGpdTpdGpdGpdGpdGpdGpdTpdGpdGpdApdCpdGpdG...
1298,2018,https://pubmed.ncbi.nlm.nih.gov/30303958/,PLoS One,https://doi.org/10.1371/journal.pone.0198980,"Pan, Q., Law, C., Yung, M., Han, K. C., Pon, Y...",ssRNA,CA72-4 A02,Homo sapien - CA72-4,5'AUCGUUACCUUAGAAGCUUCCCAAAAAGGAUUGGGGCGUCUGCA...,82,...,7.3,Not reported,"Diagnostic and Therpaeutic: "" these novel RNA ...",Not applicable,NaN,10001256,NaN,"Lau TCK, chiklau@cityu.edu.hk",https://forms.gle/n4TzuyddXQrHYJXF9,NaN
1219,2016,https://pubmed.ncbi.nlm.nih.gov/27104834/,PLoS One,https://doi.org/10.1371/journal.pone.0153637,"Marton, S., Cleto, F., Krieger, M. A., & Cardo...",ssDNA,P12-52,Escherichia coli (E. coli) ATCC 25922,5'CATACGATTTAGGTGACACTATAGCCGCCCAGCGGGGGTAGGGC...,88,...,7.4,Not reported,"Diagnostic: "" Escherichia coli is a bacterial ...",Not applicable,NaN,10001177,NaN,"Cardoso, J, josianec@gmail.com",https://forms.gle/n4TzuyddXQrHYJXF9,NaN
259,1998,https://pubmed.ncbi.nlm.nih.gov/9436913/,RNA,"PMCID: PMC1369601, PMID: 9436913","Wallace, S. T., & Schroeder, R. (1998). In vit...",ssRNA,B #84 (41 mer),Streptomycin,5'AUCACCUCAAGGAAAACGCUUCAGAAAGGGACUUAGGUGAU3',41,...,7.6,Not reported,"Drug Development: ""A growing body of evidence ...",Pb21-induced cleavage of b #84,NaN,10000240,NaN,"Schroeder, R, renee@gem.univie.ac.at",https://forms.gle/n4TzuyddXQrHYJXF9,NaN
603,2005,https://pubmed.ncbi.nlm.nih.gov/15914669/,Nucleic Acids Res,http://doi.org/10.1093/nar/gki578,"Kato, Y., Minakawa, N., Komatsu, Y., Kamiya, H...",4'-thio-RNA,CI-2-23,"Alpha-thrombin (α-thrombin), Human",5′GGGAGAAGGGAAGUAACAGGUUUA

### Работа с числовыми полями

In [59]:
df = df[df.pool_random_region != "TBD"]

Преобразуем молекулярную массу

In [60]:
def convert_to_da(value):
    if isinstance(value, (int, float)):
        return  value
    
    value = value.lower().strip()
    
    if pd.isna(value) or value in ["not reported", "n/a", "not reported"]:
        return np.nan
    
    value = str(value).replace(' ', '').strip()
    
    if 'kda' in value:
        return float(re.findall(r'\d+\.?\d*', value)[0]) * 1000
    elif 'da' in value or 'g/mol' in value:
        return float(re.findall(r'\d+\.?\d*', value)[0])
    else:
        return np.nan

In [61]:
df.molecular_weight_of_target = df.molecular_weight_of_target.apply(convert_to_da)

Преобразуем affinity

In [62]:
def affinity_value_convert(text):
    pattern = r'(\d+\.\d+|\d+)'
    
    range_match = re.match(r'Kd: (\d+\.\d+|\d+)-(\d+\.\d+|\d+) nM', text)
    if range_match:
        start_value = float(range_match.group(1))
        end_value = float(range_match.group(2))
        return (start_value + end_value) / 2
    else:
        match = re.search(pattern, text)
        if match:
            return float(match.group(0))
    return None

df.affinity = df.affinity.apply(affinity_value_convert)

Остальные поля просто преобразуем в числовой тип

In [63]:
num_columns = ["year_of_paper", "sequence_length", "gc_content", "serial_number", "ph", "affinity", "kd_nm", "pool_random_region", "molecular_weight_of_target"]

In [68]:
for col in num_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1494 entries, 0 to 1494
Data columns (total 28 columns):
 #   Column                                                                                 Non-Null Count  Dtype  
---  ------                                                                                 --------------  -----  
 0   year_of_paper                                                                          1494 non-null   int64  
 1   link_to_pubmed_entry                                                                   1494 non-null   object 
 2   journals                                                                               1494 non-null   object 
 3   journal_doi                                                                            1494 non-null   object 
 4   citation                                                                               1494 non-null   object 
 5   type_of_nucleic_acid                                                             

### Работа с категориальными столбцами

In [81]:
cat_columns = ['type_of_nucleic_acid', 'type_of_the_buffer',
               'divalent_salt']

In [82]:
df = pd.get_dummies(df, columns=cat_columns, prefix=cat_columns)

In [83]:
df.shape

(1494, 46)